In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

### 프로젝트 셋팅

In [3]:
# 학습이 완료된 모델을 저장할 파일 이름
best_model_path = 'model/best_model_segment.dat'
# 교차검증 횟수
cv_count = 5
# 교차 검증
kfold = KFold(n_splits=cv_count, shuffle=True, random_state=1)
# 평가 결과를 담을 리스트
# 필요하다면 다른 것도 만들어주세요
f1_score_list = []
# 학습 모델 이름
model_name_list = []

### 데이터 준비
- 데이터를 읽어오고 필요한 전처리까지 다 한다음 입력데이터는 train_X, 결과데이터는 train_y라는 변수에 담아서 준비해주세요

In [5]:
# 데이터 준비
df1 = pd.read_csv('data/train.csv')
df1

,ID,_2순위업종_이용금액,최대이용금액_일시불_R12M,납부_기타이용금액,쇼핑_편의점_이용금액,_2순위교통업종_이용금액,최대이용금액_CA_R12M,교통_정비이용금액,기준년월,할부금액_무이자_3M_R12M,이용개월수_온라인_R6M,연속유실적개월수_기본_24M_카드,이용금액대,청구금액_R6M,평잔_일시불_해외_6M,잔액_일시불_B2M,평잔_일시불_6M,이용카드수_신용체크,_1순위카드이용건수,Segment
0,TRAIN_000000,1408,4906,44,0,105,11710,0,201807,1314,1,13,01.100만원+,88693,0,1083,2440,1,26,D
1,TRAIN_000000,1523,5350,40,0,0,11738,0,201808,1124,1,13,01.100만원+,119419,0,736,2697,1,23,D
2,TRAIN_000000,1462,6009,44,0,0,11747,0,201809,989,1,17,01.100만원+,132269,0,997,2735,1,27,D
3,TRAIN_000000,1367,5901,48,0,0,11942,0,201810,954,1,17,01.100만원+,92304,0,697,2562,1,26,D
4,TRAIN_000000,1356,6257,44,0,0,12116,0,201811,976,0,17,01.100만원+,97254,0,607,2431,1,26,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,TRAIN_399999,2168,5760,0,0,0,0,0,201808,18391,0,2,03.30만원+,28912,0,0,1600,2,-2,E
2399996,TRAIN_399999,2687,6233,0,0,0,0,0,201809,18477,0,5,02.50만원+,42577,0,998,1742,2,-1,E
2399997,TRAIN_399999,2447,6091,0,0,0,0,0,201810,18640,0,5,03.30만원+,23976,0,814,1410,2,-2,E
2399998,TRAIN_399999,2647,4624,0,0,0,0,0,201811,15516,0,5,04.10만원+,26946,0,1485,993,2,-2,E


In [6]:
# 입력과 결과로 나눈다.
X = df1.drop(['Segment', 'ID', '기준년월'], axis=1)
y = df1['Segment']

In [7]:
# 문자열 -> 숫자
encoder1 = LabelEncoder()
encoder1.fit(y)
y2 = encoder1.transform(y)

In [8]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd

# 1. 범주형 → 수치형(Label Encoding)
X_encoded = X.copy()
for col in X_encoded.columns:
    if X_encoded[col].dtype == 'object' or X_encoded[col].dtype.name == 'category':
        le = LabelEncoder()
        X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))

# 2. 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)


In [9]:
# 학습할 데이터를 변수에 담아준다.
train_X = X_scaled
train_y = y2

### 기본 모델 사용하기
- 기본 모델 중에 만족하는 것을 찾았다면 하이퍼 파라미터 튜닝 과정은 생략하세요

In [ ]:
# SVC
svc_basic_model = SVC(probability=True)
# 교차 검증을 수행한다
r1 = cross_val_score(svc_basic_model, train_X, train_y, scoring='f1_micro', cv=kfold)
# 평가 결과를 담아준다.
f1_score_list.append(r1.mean())
# 학습 모델 이름을 담아준다.
model_name_list.append("SVC Basic")

In [ ]:
# HardVoting 구성
temp_model3 = SVC(probability=True)
#temp_model7 = GradientBoostingClassifier()

hard_voting_model_list = [
    ('model3', temp_model3)
    #('model7', temp_model7),
]

hard_voting_basic_model = VotingClassifier(estimators=hard_voting_model_list, voting='hard')

# 교차 검증을 수행한다
r1 = cross_val_score(hard_voting_basic_model, train_X, train_y, scoring='f1_micro', cv=kfold)
# 평가 결과를 담아준다.
f1_score_list.append(r1.mean())
# 학습 모델 이름을 담아준다.
model_name_list.append("HardVoting Basic")

In [ ]:
# SoftVoting 구성
temp_model3 = SVC(probability=True)
#temp_model7 = GradientBoostingClassifier()

soft_voting_model_list = [
    ('model3', temp_model3)
    #('model7', temp_model7),
]

soft_voting_basic_model = VotingClassifier(estimators=soft_voting_model_list, voting='soft')

# 교차 검증을 수행한다
r1 = cross_val_score(soft_voting_basic_model, train_X, train_y, scoring='f1_micro', cv=kfold)
# 평가 결과를 담아준다.
f1_score_list.append(r1.mean())
# 학습 모델 이름을 담아준다.
model_name_list.append("SoftVoting Basic")

In [ ]:
d1 = {
    'f1 score' : f1_score_list
}
result_df = pd.DataFrame(d1, index=model_name_list)
result_df.sort_values(by='f1 score', ascending=False, inplace=True)
result_df

In [ ]:
# 예측 결과에 대한 확률 값을 가져온다
proba_a1 = lr_grid_clf.best_estimator_.predict_proba(train_X)
# 0일 확률
a10 = proba_a1[:, 0]
# 1일 확률
a20 = proba_a1[:, 1]

plt.scatter(list(range(len(a10))), a10, label='0일 확률')
plt.scatter(list(range(len(a20))), a20, label='1일 확률')
plt.ylim(-0.1, 1.1)
plt.legend()
plt.show()

In [ ]:
# 예측 결과에 대한 확률 값을 가져온다
proba_a1 = xgboost_grid_clf.best_estimator_.predict_proba(train_X)
# 0일 확률
a10 = proba_a1[:, 0]
# 1일 확률
a20 = proba_a1[:, 1]

plt.scatter(list(range(len(a10))), a10, label='0일 확률')
plt.scatter(list(range(len(a20))), a20, label='1일 확률')
plt.ylim(-0.1, 1.1)
plt.legend()
plt.show()

In [ ]:
# 예측 결과에 대한 확률 값을 가져온다
proba_a1 = ada_grid_clf.best_estimator_.predict_proba(train_X)
# 0일 확률
a10 = proba_a1[:, 0]
# 1일 확률
a20 = proba_a1[:, 1]

plt.scatter(list(range(len(a10))), a10, label='0일 확률')
plt.scatter(list(range(len(a20))), a20, label='1일 확률')
plt.ylim(-0.1, 1.1)
plt.legend()
plt.show()

In [ ]:
10 / 0

In [ ]:
# 최종 모델을 생성하고 전체 데이터를 학습 시킨다.
best_model = XGBClassifier()
best_model.set_params(**xgboost_grid_clf.best_params_)
best_model.fit(train_X, train_y)
best_model

In [ ]:
# 학습 모델 등을 저장한다.
with open(best_model_path, 'wb') as fp :
    pickle.dump(best_model, fp)
    pickle.dump(encoder1, fp)
    pickle.dump(scaler1, fp)

print('저장완료')

In [ ]:
# 저장 확인
with open(best_model_path, 'rb') as fp :
    test_model = pickle.load(fp)
    test_encoder = pickle.load(fp)
    test_scaler1 = pickle.load(fp)

In [ ]:
r1 = test_model.predict(train_X)
r2 = f1_score(train_y, r1)
r2